In [96]:
#!pip install pyserini -q
#!pip install faiss-gpu -q
##!pip install evaluate -q
##!pip install trectools -q
##!pip install jsonlines -q
##!pip install datasets -q
!pip install sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [120]:
import json
import re
from time import time

import pandas as pd
import numpy as np


from tqdm import tqdm
import spacy

import pickle


from pyserini.search.lucene import LuceneSearcher
from sentence_transformers import CrossEncoder

In [1]:
import os
os.chdir('/content/drive/MyDrive/doutorado/P_IA368DD_2023S/aula10_11')

In [ ]:
'''os.makedirs(f'data', exist_ok=True)

!wget http://jamesf-incomplete-qa.s3.amazonaws.com/iirc.tar.gz
!tar -xzf iirc.tar.gz
!rm iirc.tar.gz
!mv iirc data
#!cd iirc

!wget http://jamesf-incomplete-qa.s3.amazonaws.com/context_articles.tar.gz
!tar -xzf context_articles.tar.gz
!rm context_articles.tar.gz
!

!wget https://iirc-dataset.s3.us-west-2.amazonaws.com/iirc_test.json 
!mv iirc_test.json data/iirc/
'''

In [10]:
def remove_html(html):
  # Acrescentei replace '\xa0' 
  return re.sub("<[^>]*>", "", html).strip()

In [7]:
path_data = "./data"

dev_set = json.load(open(f'{path_data}/iirc/dev.json','r'))
test_set = json.load(open(f'{path_data}/iirc/iirc_test.json', 'r'))
context_articles = json.load(open(f"{path_data}/context_articles.json",'r'))

In [11]:
for titulo in context_articles.keys():
  context_articles[titulo] = remove_html(context_articles[titulo])

In [13]:
%time
documents = []
all_titles = []

for item in test_set:
    if item['title'].lower() not in all_titles:
        documents.append({
                "title": item['title'],
                "content": item["text"]
            }
        )
        all_titles.append(item['title'].lower())
    for link in item["links"]:
        if link['target'].lower() in context_articles and link['target'].lower() not in all_titles:
            documents.append({
                "title": link['target'],
                "content": context_articles[link['target'].lower()]
            })
            all_titles.append(link['target'].lower())

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 14.5 µs


In [111]:
corpus_list = []

# Se for usar só os documentos que estão na base de teste
for doc in documents:
  #break;
  titulo = doc['title']
  conteudo = doc['content']
  corpus_list.append({"title": titulo, "content": conteudo})
  #artigos_sem_html.append({"title": titulo, "content": remove_html(conteudo)})


In [23]:

nlp = spacy.blank("en")
nlp.add_pipe("sentencizer")

stride = 2
max_length = 3

def window(documents, 
           stride=2, 
           #stride=3, #without overlap
           max_length=3):
    treated_documents = []

    for j,document in enumerate(tqdm(documents)):
        doc_text = document['content']
        doc = nlp(doc_text)
        sentences = [sent.text.strip() for sent in doc.sents]
        for i in range(0, len(sentences), stride):
            segment = ' '.join(sentences[i:i + max_length])
            treated_documents.append({
                "title": document['title'],
                "contents": document['title']+". "+segment,
                "segment": segment
            })
            if i + max_length >= len(sentences):
                break
                
    return treated_documents

treated_documents = window(documents)

100%|██████████| 7028/7028 [02:34<00:00, 45.43it/s]


In [26]:
#if gerar_artigos_segmentados:
artigos_segmentados = treated_documents
with open(f'{path_data}/artigos_segmentados.pickle', 'wb') as f:
  pickle.dump(artigos_segmentados, f)

df_artigos_segmentados = pd.DataFrame(artigos_segmentados)
df_artigos_segmentados

In [28]:
os.makedirs(f'{path_data}/iirc/iirc_indices', exist_ok=True)

f = open(f'{path_data}/iirc/iirc_indices/contents.jsonl','w')
for i, doc in enumerate(treated_documents):
    doc['id'] = i
    if doc['segment'] != "":
        f.write(json.dumps(doc)+"\n")

In [31]:
%%time
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input {path_data}/iirc/iirc_indices \
  --index {path_data}/iirc/iirc_index \
  --generator DefaultLuceneDocumentGenerator \
  --threads 9 \
  --storePositions --storeDocvectors --storeRaw

2023-05-17 23:11:53,123 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2023-05-17 23:11:53,125 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2023-05-17 23:11:53,125 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2023-05-17 23:11:53,126 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: ./data/iirc/iirc_indices
2023-05-17 23:11:53,126 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2023-05-17 23:11:53,127 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2023-05-17 23:11:53,127 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 9
2023-05-17 23:11:53,128 INFO  [main] index.IndexCollection (IndexCollection.java:389) - Language: en
2023-05-17 23:11:53,128 INFO  [main] index.IndexCollectio

In [33]:
searcher = LuceneSearcher(f'{path_data}/iirc/iirc_index')

In [191]:
n_examples = 50
np.random.seed(42)

#samples_index = np.random.randint(0,len(test_set),n_examples)
all_dicio = []
all_q = []
for item in test_set:
  question = item['questions']
  if question[0]['answer']['type'] not in ['value','binary']:
    continue
  else:
    dicio = {}
    q = question[0]
    for cnt, context in enumerate(q['context']): 
      dicio['title'] = item['title']
      dicio['question'] = q['question']
      dicio['text'] = context['text'].replace('\n','') #item['text']
      dicio['passage'] = context['passage'].replace('\n','')
      dicio['answer_type'] = q['answer']['type']
      dicio['answer_value'] = q['answer']['answer_value']
    all_dicio.append(dicio)

sampled_q = np.random.choice(all_dicio, n_examples)
df_sampled_q = pd.DataFrame(sampled_q)

In [ ]:
df_sampled_q = None#pd.DataFrame({})
for i in sampled_q:
  if df_sampled_q is None:
    df_sampled_q = pd.DataFrame([i])
  else:
    aux = pd.DataFrame([i])
    df_sampled_q = pd.concat((df_sampled_q,aux),axis=0)
df_sampled_q = df_sampled_q.reset_index(drop=True)
df_sampled_q.to_csv(f"{path_data}/list_prompt_answer_ground_truth.csv")
df_sampled_q

In [98]:
model_id = "cross-encoder/ms-marco-MiniLM-L-6-v2"
model_name = "ms-marco-MiniLM-L-6-v2"
model = CrossEncoder(model_id, max_length=512, num_labels=1)
#model = model.to(device)

In [207]:
def reranking_cross_encoder(docs, max=1000, batch_size=500):
    scores = []

    query_ids = []
    passage_ids = []

    start = time()
    for i in tqdm(range(0, len(docs), batch_size), leave=False):  # tqdm(docs):
        i_end = i + batch_size
        i_end = len(docs) if i_end > len(docs) else i_end

        batch = docs[i:i_end]
        
        text_pairs = [(sample['query_text'], sample['passage_text']) for sample in batch]
        #print(text_pairs[0])
        #print(text_pairs[1])
        
        predictions = model.predict(text_pairs)
        
        for score, result in zip(predictions, batch):
            query_id = result["query_id"]
            query_ids.append(query_id)

            passage_id = result["passage_id"]
            passage_ids.append(passage_id)
            scores.append(score)

    results = []

    for query, passage, score in zip(query_ids, passage_ids, scores):
        results.append((query, passage, score))

    sorted_list = sorted(results, key=lambda x: x[2], reverse=True)
    end = time()
    elapsed_time = round(end - start, 3)

    #if "cross_encoder" not in data:
    #    data["cross_encoder"] = {
    #        "reranking": 0
    #    }
    data = {"cross_encoder": {"reranking": 0}}
    data["cross_encoder"]["reranking"] += elapsed_time

    return sorted_list[:max], data

def run_pipeline(df_amostra, passages,searcher, top_k = 1):
    docs_reranking = []
    #for id, query in tqdm(zip(test_ids, test_corpus)):
    for index, question in tqdm(df_amostra.iterrows()):
      bm25_hits = searcher.search(question['text'], k = top_k)
      docs = []
      for doc_id in bm25_hits:
        #if int(doc_id.docid) > len(passages):
        #  continue
        #print('----- ',int(doc_id.docid))
        #passagens = (question['text'], passages[int(doc_id.docid)]['content'])# for docid in docids]
        passagens = passages[passages['id']== int(doc_id.docid)]["contents"].values
        if len(passagens) >= 1:
          passagens = passagens[0]
        else:
          import pdb;
          pdb.set_trace();

        doc = {
            "query_id": index,
            "passage_id": doc_id.docid,
            "query_text": question['text'],
            "passage_text": passagens,
        }
        docs.append(doc)
      # Second stage
      #print("here")
      aux, _dt = reranking_cross_encoder(docs, 100, 100)
      docs_reranking.extend(aux)
    return docs_reranking, _dt

In [ ]:
docs_reranking, data = run_pipeline(df_amostra=df_sampled_q,passages=df_artigos_segmentados,searcher=searcher,)

In [ ]:
#ranks = np.arange(1,1)
all_ranks = np.tile([1],50)
df_docs_reranking = pd.DataFrame(docs_reranking,columns=["query","docid","score"])
df_docs_reranking['docid'] = df_docs_reranking['docid'].astype(int)
df_docs_reranking['q0'] = "Q0"
df_docs_reranking["rank"] = all_ranks#.reshape(-1,1)
df_docs_reranking["system"] = "CrossEncoder"

copy_df_artigos_segmentados = df_artigos_segmentados.copy()
copy_df_artigos_segmentados = copy_df_artigos_segmentados.rename(columns={'id':'docid'})

copy_df_sampled_q = df_sampled_q.copy()
copy_df_sampled_q['query'] = copy_df_sampled_q.index

df_docs_reranking = df_docs_reranking.merge(copy_df_artigos_segmentados[['title','contents','docid']],on='docid',how='left')
df_docs_reranking = df_docs_reranking.merge(copy_df_sampled_q[['answer_type','answer_value','query']],on='query',how='left')


df_docs_reranking

In [236]:
df_docs_reranking.to_csv(f"{path_data}/list_prompt_answer_search.csv")